# Implantação de Modelo de Machine Learning com Azure ML 🚀🔍

Este notebook demonstra o processo de implantação de um modelo treinado usando o Azure Machine Learning. Exploraremos como salvar o modelo, registrar na plataforma e publicar um endpoint para servir o modelo em produção.


In [ ]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
import joblib


In [ ]:
# Conecte ao workspace do Azure ML
ws = Workspace.from_config()
print("Conectado ao Workspace:", ws.name)


In [ ]:
# Carregar o modelo
model = joblib.load("model.pkl")


In [ ]:
# Registrar o modelo no workspace
model = Model.register(workspace=ws,
                       model_path="model.pkl",  # Caminho local do modelo
                       model_name="modelo_exemplo")  # Nome para o modelo registrado
print("Modelo registrado:", model.name)


In [ ]:
#Defina o ambiente, incluindo as bibliotecas necessárias para executar o modelo. Isso pode ser feito criando um Environment personalizado ou usando um pré-definido.
env = Environment(name="env-exemplo")
env.docker.enabled = True
env.python.conda_dependencies.add_pip_package("scikit-learn")


In [ ]:
%%writefile score.py

import joblib
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def init():
    global model
    model = joblib.load("model.pkl")

def run(data):
    try:
        data = np.array(json.loads(data))
        prediction = model.predict(data)
        return json.dumps({"result": prediction.tolist()})
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})


In [ ]:
inference_config = InferenceConfig(entry_script="score.py", environment=env)


In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace=ws,
                       name="modelo-deploy-aci",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)
print("Modelo implantado no endpoint:", service.scoring_uri)


In [ ]:
#Faça uma solicitação de teste ao endpoint para verificar a inferência.
python
Copy code

import json
import requests

# Substitua com um exemplo de dados de entrada
input_data = json.dumps({"data": [[5.1, 3.5, 1.4, 0.2]]})  # Exemplo de dados
headers = {'Content-Type': 'application/json'}

response = requests.post(service.scoring_uri, data=input_data, headers=headers)
print("Resposta do modelo:", response.json())


## Conclusões

O modelo foi implantado com sucesso usando Azure ML e um endpoint foi publicado para receber requisições de inferência. Esse processo de implantação garante que o modelo esteja disponível para uso em produção, e futuras melhorias podem incluir ajustes no ambiente de implantação e monitoramento de métricas.
